## Декораторы

In [3]:
def say_hello(name):
    return f"Привет, {name}!"

def howare(name):
    return f"Эй, {name}, как у тебя дела?"

def greet_margo(greeter_func):
    return greeter_func("Маргарита")

In [4]:
greet_margo(say_hello)

'Привет, Маргарита!'

In [5]:
greet_margo(howare)

'Эй, Маргарита, как у тебя дела?'

Функции, определенные внутри других функций, называются внутренними (inner functions). Пример функции с двумя внутренними функциями:

In [6]:
def parent():
    print("Привет из функции parent().")

    def first_child():
        print("Привет из функции first_child().")

    def second_child():
        print("Привет из функции second_child().")

    second_child()
    first_child()

In [7]:
parent()

Привет из функции parent().
Привет из функции second_child().
Привет из функции first_child().


Внутренние функции не определены, пока не вызвана родительская функция. То есть они локально ограничены parent() и существуют только внутри нее, как локальные переменные. При вызове функции first_child() за пределами parent() мы получим ошибку:

In [8]:
first_child()

NameError: name 'first_child' is not defined

Python позволяет использовать функции в качестве возвращаемых значений. В следующем примере возвращается одна из внутренних функций внешней функции parent():

In [11]:
def parent(num):
    def first_child():
        return "Привет, меня зовут Маргарита."

    def second_child():
        return "Зови меня Мария."

    if num == 1:
        return first_child
    else:
        return second_child

In [13]:
first = parent(1)
first()

'Привет, меня зовут Маргарита.'

## Простые декораторы

In [14]:
def my_decorator(func):
    def wrapper():
        print("До вызова функции.")
        func()
        print("После вызова функции.")
    return wrapper

def say_whee():
    print("Ура!")

say_whee = my_decorator(say_whee)

In [15]:
say_whee()

До вызова функции.
Ура!
После вызова функции.


In [16]:
def not_during_the_night(func):
    def wrapper():
        if 8 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Тише, соседи спят!
    return wrapper

def say_whee():
    print("Ура!")

say_whee = not_during_the_night(say_whee)

In [17]:
def my_decorator(func):
    def wrapper():
        print("До вызова функции.")
        func()
        print("После вызова функции.")
    return wrapper

@my_decorator
def say_whee():
    print("Ура!")

In [18]:
say_whee()

До вызова функции.
Ура!
После вызова функции.


### Примеры

In [28]:
import functools
import time

def timer(func):
    """Выводит время выполнения декорируемой функции"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter() 
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Функция {func.__name__!r} выполнена за {run_time:.4f} с")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [29]:
waste_some_time(999)

Функция 'waste_some_time' выполнена за 3.2160 с


In [30]:
import functools

def debug(func):
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Вызываем {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} возвращает {value!r}")         # 4
        return value
    return wrapper_debug

In [32]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Привет, {name}!"
    else:
        return f"Ого, {name}! Тебе уже {age}, как ты быстро растёшь!"

In [33]:
make_greeting("Маргарита")

Вызываем make_greeting('Маргарита')
'make_greeting' возвращает 'Привет, Маргарита!'


'Привет, Маргарита!'

In [35]:
import math
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

In [36]:
approximate_e(5)

Вызываем factorial(0)
'factorial' возвращает 1
Вызываем factorial(1)
'factorial' возвращает 1
Вызываем factorial(2)
'factorial' возвращает 2
Вызываем factorial(3)
'factorial' возвращает 6
Вызываем factorial(4)
'factorial' возвращает 24


2.708333333333333

In [37]:
import time

def slow_down(func):
    """Ждёт 1 секунду, прежде чем вызвать переданную функцию"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Поехали!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [38]:
countdown(3)

3
2
1
Поехали!
